# 📁 Projet final – Data Crafting & Data Mining

Ce projet vous permettra de mettre en œuvre tout le cycle de traitement de données :
- Qualité et structuration (Data Crafting)
- Exploration, préparation et modélisation (Data Mining)

**Domaine :** Gestion des ressources humaines (1200 employés)

**Choix de modélisation :** 2 modélisations possibles à la partie 5. Faites votre choix et poursuivez le projet
- Classification : Construire un modèle prédictif du type de contrat et Évaluer la précision sur les nouveaux employés
- Clustering : Identifier naturellement des profils d’employés et Interpréter les groupes pour recommandations RH

**Fichier de données :** `dataset_rh.csv`

📅 Délai de rendu : 1 semaine après la fin du module.

##  Partie 1 – Chargement & exploration initiale

In [2]:
import great_expectations as ge
import pandas as pd

# Charger le dataset
df = pd.read_csv("dataset_rh.csv")

# TODO : Afficher les 5 premières lignes et les dimensions du dataset
# ...
print(df.shape)
df.head()

(1200, 11)


,EmployeID,Nom,Prenom,Age,Sexe,Departement,Poste,Salaire,NiveauEtude,TypeContrat,DateEmbauche
0,EMP00000,Dijoux,Zacharie,36,F,Marketing,Chargé de com,21649.44,Bac+3,Stage,2017-03-17
1,EMP00001,Philippe,Théophile,64,H,Marketing,Chargé de com,22127.03,Bac+3,CDI,NaN
2,EMP00002,Blondel,Catherine,53,H,Production,Comptable,26105.59,Bac+3,Alternance,2023-05-02
3,EMP00003,Berthelot,Maryse,47,H,Marketing,Chef de projet,29066.15,Master,CDD,2016-03-24
4,EMP00004,Clément,Christelle,25,F,Informatique,Développeur,34440.50,Bac+2,CDI,2016-09-04


##  Partie 2 – nettoyage, alignement, validation puis pipeline

In [ ]:
# TODO :
# - Identifier les colonnes avec des valeurs manquantes ou corrompues
# - Corriger les âges aberrants, les salaires négatifs
# - Nettoyer les dates incorrectes (ex: "Non précisé") et convertir les dates valides
# - Supprimer ou corriger les doublons éventuels
# - Nettoyer les formats (genre, contrat, niveau d'études)
# - Utiliser great Expectations pour la validation
# - Créer un pipeline complet de cette première étape

# Identifier les colonnes avec des valeurs manquantes ou corrompues
print(df.isnull())

def correct_age(df):
    # Corriger les âges aberrants en mettant la mediane des âges valides si l'âge est en dehors de 18-65
    median_age = df["Age"].median()
    df["Age"] = df["Age"].apply(lambda x: median_age if x < 18 or x > 65 else x)
    return df

def correct_salary(df):
    # Corriger les salaires négatifs en mettant la mediane des salaires valides
    median_salary = df["Salaire"].median()
    df["Salaire"] = df["Salaire"].apply(lambda x: median_salary if x < 0 else x)
    return df

def clean_dates(df):
    # Nettoyer les dates incorrectes et convertir les dates valides
    df["DateEmbauche"] = pd.to_datetime(df["DateEmbauche"], errors="coerce")
    df["DateEmbauche"].fillna(df["DateEmbauche"].median(), inplace=True)
    return df

def remove_duplicates(df):
    # Supprimer les doublons éventuels
    df = df.drop_duplicates()
    return df

def clean_formats(df):
    # Nettoyer les formats (genre, contrat, niveau d'études)
    df["Sexe"] = df["Sexe"].str.strip().str.capitalize()
    # we might have values outside of H or F, if so, we can set them to 'A'
    df["Sexe"] = df["Sexe"].apply(lambda x: x if x in ["H", "F"] else "A")
    df["TypeContrat"] = df["TypeContrat"].str.strip().str.upper()
    df["NiveauEtude"] = df["NiveauEtude"].str.strip().str.capitalize()
    return df


def validate_with_great_expectations(df):
    # EmployeID,Nom,Prenom,Age,Sexe,Departement,Poste,Salaire,NiveauEtude,TypeContrat,DateEmbauche
    context = ge.get_context(mode="ephemeral")
    datasource = context.data_sources.add_pandas("pandas_datasource")
    data_asset = datasource.add_dataframe_asset(name="rh_data")
    batch_definition = data_asset.add_batch_definition_whole_dataframe("batch_def")

    # --- Définition du suite d'expectations ---
    expectations = ge.ExpectationSuite(name="rh_validation_suite")

    # EmployeID
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToNotBeNull(column="EmployeID")
    )
    # Nom et Prenom
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToNotBeNull(column="Nom")
    )
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToNotBeNull(column="Prenom")
    )
    # Sexe
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToNotBeNull(column="Sexe")
    )
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToBeInSet(
            column="Sexe", value_set=["H", "F", "A"]
        )
    )
    # Departement
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToNotBeNull(column="Departement")
    )
    # Poste
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToNotBeNull(column="Poste")
    )
    # Salaire
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToNotBeNull(column="Salaire")
    )
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToBeBetween(column="Salaire", min_value=0)
    )
    # NiveauEtude
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToNotBeNull(column="NiveauEtude")
    )
    # TypeContrat
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToNotBeNull(column="TypeContrat")
    )
    # DateEmbauche
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToNotBeNull(column="DateEmbauche")
    )
    expectations.add_expectation(
        ge.expectations.ExpectColumnValuesToBeOfType(
            column="DateEmbauche", type_="datetime64[ns]"
        )
    )

    # On enregistre le suite dans le contexte
    expectations = context.suites.add(expectations)

    # ⚠️ ICI était l'erreur : il faut utiliser `data=` et `suite=`
    validation_definition = ge.ValidationDefinition(
        name="rh_validation_definition",
        data=batch_definition,  # et non batch_definition=...
        suite=expectations,  # et non expectation_suite=...
    )
    validation_definition = context.validation_definitions.add(validation_definition)

    results = validation_definition.run(batch_parameters={"dataframe": df})

    print(f"\n  Validation réussie: {results.success}")
    print(f"  Nombre de tests: {len(results.results)}")
    for result in results.results:
        status = "OK" if result.success else "ECHEC"
        print(f"    [{status}] {result.expectation_config.type} on {result.expectation_config.kwargs}")

    return df


def data_cleaning_pipeline(df):
    df = correct_age(df)
    df = correct_salary(df)
    df = clean_dates(df)
    df = remove_duplicates(df)
    df = clean_formats(df)
    df = validate_with_great_expectations(df)
    return df

df = data_cleaning_pipeline(df)

C:\Users\guill\AppData\Local\Temp\ipykernel_11132\3625523461.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["DateEmbauche"].fillna(df["DateEmbauche"].median(), inplace=True)


      EmployeID    Nom  Prenom    Age   Sexe  Departement  Poste  Salaire  \
0         False  False   False  False  False        False  False    False   
1         False  False   False  False  False        False  False    False   
2         False  False   False  False  False        False  False    False   
3         False  False   False  False  False        False  False    False   
4         False  False   False  False  False        False  False    False   
...         ...    ...     ...    ...    ...          ...    ...      ...   
1195      False  False   False  False  False        False  False    False   
1196      False  False   False  False  False        False  False    False   
1197      False  False   False  False  False        False  False    False   
1198      False  False   False  False  False        False  False    False   
1199      False  False   False  False  False        False  False    False   

      NiveauEtude  TypeContrat  DateEmbauche  
0           False        Fal

Calculating Metrics: 100%|██████████| 68/68 [00:00<00:00, 5116.38it/s]


  Validation réussie: False


  Nombre de tests: 13
    [OK] expect_column_values_to_not_be_null
    [OK] expect_column_values_to_not_be_null
    [OK] expect_column_values_to_not_be_null
    [OK] expect_column_values_to_not_be_null
    [OK] expect_column_values_to_be_in_set
    [OK] expect_column_values_to_not_be_null
    [OK] expect_column_values_to_not_be_null
    [OK] expect_column_values_to_not_be_null
    [OK] expect_column_values_to_be_between
    [ECHEC] expect_column_values_to_not_be_null
    [ECHEC] expect_column_values_to_not_be_null
    [OK] expect_column_values_to_not_be_null
    [OK] expect_column_values_to_be_of_type


## Partie 3 – Construction du pipeline de transformation

In [4]:
# TODO :
# - Utiliser le pipeline précédent pour créer un DataFrame nettoyé
# - Encodage des colonnes catégorielles
# - Normalisation éventuelle de variables numériques
# - Vérification finale de la complétude
# - créer un pipeline final = pipeline nettayage + pipeline transformation 


## Partie 4 – Exploration statistique

In [5]:
# TODO :
# - Visualiser la répartition des âges, salaires, types de contrat
# - Identifier des corrélations ou anomalies intéressantes
# - Produire des graphiques utiles à l'analyse RH


## Partie 5 – Modélisation (clustering ou classification)
Pour la suite du projet vous êtes libres de choisir entre les 2 problèmes:
- Classification : Construire un modèle prédictif du type de contrat et Évaluer la précision sur les nouveaux employés
- Clustering : Identifier naturellement des profils d’employés et Interpréter les groupes pour recommandations RH

In [6]:
# TODO :
# - Appliquer un algorithme de ML (KMeans, LogisticRegression, DecisionTree…)
# - Évaluer les résultats obtenus (visualisation ou métriques)


## Partie 6 – Conclusion et recommandations

In [7]:
# TODO :
# - Synthétiser les problèmes détectés et les corrections apportées
# - Présenter une recommandation métier à partir des résultats
